I refered to OpenAI CookBook 'Assistants_API_overview_python.ipynb' and made modifications

This notebook shows my attempt to use an existing OpenAI Assistant API with function calling using Python~
1. Create/Use an Assistant
2. Create/Use a Thread
3. Run Thread with Assistant
4. Define functions
5. Execute function and get response from the Assistant.

Earlier on, I have created an Assistant API called "chettybot" in OpenAI Playground. 
Below is the chettybot's id. Create your own :D

## Assistants API

- `Assistants`, which encapsulate a base model, instructions, tools, and (context) documents,
- `Threads`, which represent the state of a conversation, and
- `Runs`, which power the execution of an `Assistant` on a `Thread`, including textual responses and multi-step tool use.

In [86]:
from dotenv import load_dotenv
from utils.modules import *
load_dotenv() # Load .env file
from openai import OpenAI
client = OpenAI() # Initialize OpenAI Client

In [87]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [88]:
# Use existing, or create a new one
get_existing_assistant = False #set true if want to use existing assistant
get_previous_thread = False #set true if want to use existing thread

assistant_id_to_use = "asst_b11XnU3yNtM3CHOOoNKNYOvt" #ieol
thread_id_to_use = "thread_EbHkQvVh82Ubzfk3eqtmSsVv" #ieol

- Change the flag if u want to create a new assistant.

In [89]:
if get_existing_assistant:
    assistant = get_assistant(client, assistant_id_to_use) # Retrieve Assistant
    print(assistant.name + " is ready~~")
else:
    name = "ChettyBot2"
    description = "A chatbot for fun."
    instructions = "You are a chatty bot who can search weather and search animal. Entertain the user."
    tools = [{'function': {'name': 'search_weather',
    'parameters': {'title': 'WeatherSearch',
     'description': 'Call this to get the weather at that location',
     'type': 'object',
     'properties': {'location': {'title': 'Location',
       'description': 'location to get weather for',
       'type': 'string'}},
     'required': ['location']},
    'description': 'Run weather search.'},
   'type': 'function'},
  {'function': {'name': 'search_animal',
    'parameters': {'title': 'AnimalSearch',
     'description': 'Call this to get the animal',
     'type': 'object',
     'properties': {'animal': {'title': 'Animal',
       'description': 'animal that is mentioned',
       'type': 'string'}},
     'required': ['animal']},
    'description': 'Run animal search.'},
   'type': 'function'}]
    assistant = create_assistant(client, name, description, instructions) # Create Assistant
    print("New Assistant created with ID: " + assistant.id)

New Assistant created with ID: asst_3qO0sIuWKrdJxSa6qqbVpApV


In [31]:
show_json(assistant)

{'id': 'asst_b11XnU3yNtM3CHOOoNKNYOvt',
 'created_at': 1702605089,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a chatty bot who can search weather and search animal. Entertain the user.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'chettybot',
 'object': 'assistant',
 'tools': [{'function': {'name': 'search_weather',
    'parameters': {'title': 'WeatherSearch',
     'description': 'Call this to get the weather at that location',
     'type': 'object',
     'properties': {'location': {'title': 'Location',
       'description': 'location to get weather for',
       'type': 'string'}},
     'required': ['location']},
    'description': 'Run weather search.'},
   'type': 'function'},
  {'function': {'name': 'search_animal',
    'parameters': {'title': 'AnimalSearch',
     'description': 'Call this to get the animal',
     'type': 'object',
     'properties': {'animal': {'title': 'Animal',
       'description': 'animal that is mentioned',
       'type': '

In [32]:
# Retrieve the previous conversation thread
if get_previous_thread:
    thread = get_chat(client, thread_id_to_use)
    print("Chat retrieved with ID: " + thread.id)
    print(thread)
else:
    thread = start_new_chat(client)
    print("New chat created with ID: " + thread.id)

New chat created with ID: thread_vqX9Bsk254oL4jIrH9bFDmxs


In [33]:
show_json(thread)

{'id': 'thread_vqX9Bsk254oL4jIrH9bFDmxs',
 'created_at': 1702882464,
 'metadata': {},
 'object': 'thread'}

### Create the thread independently (without any assistant)

In [34]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is your name?",
)
show_json(message)

{'id': 'msg_rh8mQvR9Xoq5vAep8ifaciI0',
 'assistant_id': None,
 'content': [{'text': {'annotations': [], 'value': 'What is your name?'},
   'type': 'text'}],
 'created_at': 1702882465,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_vqX9Bsk254oL4jIrH9bFDmxs'}

This shows that a thread can exist independently without an Assistant. 
- This is useful so that we can allow any Assistant to look at the messages in the thread.

### Run the thread with the Assistant
map assistant_id=assistant.id 

In [35]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_rV9JGHyHXqphPtfdolyGH065',
 'assistant_id': 'asst_b11XnU3yNtM3CHOOoNKNYOvt',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702882467,
 'expires_at': 1702883067,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a chatty bot who can search weather and search animal. Entertain the user.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_vqX9Bsk254oL4jIrH9bFDmxs',
 'tools': [{'function': {'name': 'search_weather',
    'parameters': {'title': 'WeatherSearch',
     'description': 'Call this to get the weather at that location',
     'type': 'object',
     'properties': {'location': {'title': 'Location',
       'description': 'location to get weather for',
       'type': 'string'}},
     'required': ['location']},
    'description': 'Run weather search.'},
   'type': 'function'},
  {'function': {'name': 'search_animal

Now, the thread is being attached to the Assistant. Observe the 'assistant_id', and notice that the status is 'queued'. 
No messages is being sent yet at this point.

To get a completion from an Assistant for a given Thread, we must create a Run. Creating a Run will indicate to an Assistant it should look at the messages in the Thread and take action: either by adding a single response, or using tools.

> **Note**
> Runs are a key difference between the Assistants API and Chat Completions API. While in Chat Completions the model will only ever respond with a single message, in the Assistants API a Run may result in an Assistant using one or multiple tools, and potentially adding multiple messages to the Thread.

To get our Assistant to respond to the user, let's create the Run. As mentioned earlier, you must specify _both_ the Assistant and the Thread.


### Run

In [36]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [37]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_rV9JGHyHXqphPtfdolyGH065',
 'assistant_id': 'asst_b11XnU3yNtM3CHOOoNKNYOvt',
 'cancelled_at': None,
 'completed_at': 1702882469,
 'created_at': 1702882467,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a chatty bot who can search weather and search animal. Entertain the user.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1702882467,
 'status': 'completed',
 'thread_id': 'thread_vqX9Bsk254oL4jIrH9bFDmxs',
 'tools': [{'function': {'name': 'search_weather',
    'parameters': {'title': 'WeatherSearch',
     'description': 'Call this to get the weather at that location',
     'type': 'object',
     'properties': {'location': {'title': 'Location',
       'description': 'location to get weather for',
       'type': 'string'}},
     'required': ['location']},
    'description': 'Run weather search.'},
   'type': 'function'},
  {'function': {'name': 'sear

### Messages
Now that the Run has completed, we can list the Messages in the Thread to see what got added by the Assistant.

In [38]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_f163RNptFeZhNSRXnCncD321',
   'assistant_id': 'asst_b11XnU3yNtM3CHOOoNKNYOvt',
   'content': [{'text': {'annotations': [],
      'value': 'Hello there! You can call me Chatty Bot. What can I do for you today?'},
     'type': 'text'}],
   'created_at': 1702882468,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_rV9JGHyHXqphPtfdolyGH065',
   'thread_id': 'thread_vqX9Bsk254oL4jIrH9bFDmxs'},
  {'id': 'msg_rh8mQvR9Xoq5vAep8ifaciI0',
   'assistant_id': None,
   'content': [{'text': {'annotations': [], 'value': 'What is your name?'},
     'type': 'text'}],
   'created_at': 1702882465,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_vqX9Bsk254oL4jIrH9bFDmxs'}],
 'object': 'list',
 'first_id': 'msg_f163RNptFeZhNSRXnCncD321',
 'last_id': 'msg_rh8mQvR9Xoq5vAep8ifaciI0',
 'has_more': False}

Above is showing the history of the conversation (this is called 'thread'). 
- Notice that the latest message is at the top. This helps because you can always just choose to print the *very first* element of the content collection. a.k.a `content[0].text.value`.
- Wtv it is, you can still change the sort order to "desc" if you want to~

In [39]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_tbsVS4FTraNcNnVNjF0mNXyx',
   'assistant_id': 'asst_b11XnU3yNtM3CHOOoNKNYOvt',
   'content': [{'text': {'annotations': [],
      'value': "Of course, I'd be happy to explain it to you! What specifically would you like me to explain?"},
     'type': 'text'}],
   'created_at': 1702882477,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_CPXO3iDFRg4d7EBUpeUAoPCv',
   'thread_id': 'thread_vqX9Bsk254oL4jIrH9bFDmxs'}],
 'object': 'list',
 'first_id': 'msg_tbsVS4FTraNcNnVNjF0mNXyx',
 'last_id': 'msg_tbsVS4FTraNcNnVNjF0mNXyx',
 'has_more': False}

Notice: It can be a bad request if the assistant is waiting for your response (e.g if they use function calling, and is waiting for ur input). You cannot run a new message if this occurs. 

## Example

- `submit_message`: create a Message on a Thread, then start (and return) a new Run
- `get_response`: returns the list of Messages in a Thread

In [40]:
from openai import OpenAI

ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

client = OpenAI()

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

I've also defined a `create_thread_and_run` function that I can re-use (which is actually almost identical to the [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) compound function in our API ;) ).

In [43]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "What are you capable of?"
)
thread2, run2 = create_thread_and_run("I don't like you. What can I do?")

# Now all Runs are executing...

In [44]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Thank our assistant on Thread 3 :)
run3 = submit_message(ASSISTANT_ID, thread2, "Thank you!")
run3 = wait_on_run(run3, thread2)
pretty_print(get_response(thread2))

# Messages
user: What are you capable of?
assistant: I can search for the weather and look up information about animals! Just let me know what you're curious about, and I'll be happy to help.

# Messages
user: I don't like you. What can I do?
assistant: That's okay! You can ask me to search for the weather in a specific location, or you can ask me to search for an animal. I'm here to help with whatever you need!

# Messages
user: I don't like you. What can I do?
assistant: That's okay! You can ask me to search for the weather in a specific location, or you can ask me to search for an animal. I'm here to help with whatever you need!
user: Thank you!
assistant: You're welcome! Feel free to ask me anything, I'm here to help entertain and assist you.



# Function Calling

At below, (`'tools': [{'function': {'name':`) you can see that the Assistant has 2 functions listed; 'search_weather' & 'search_animal'. 
Assistant should be able to identify which function to call, depending on the User's input.
- Once the Assistant identify the appropriate function to call, it requires YOUR action to execute it.

In [45]:
show_json(assistant)

{'id': 'asst_b11XnU3yNtM3CHOOoNKNYOvt',
 'created_at': 1702605089,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a chatty bot who can search weather and search animal. Entertain the user.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'chettybot',
 'object': 'assistant',
 'tools': [{'function': {'name': 'search_weather',
    'parameters': {'title': 'WeatherSearch',
     'description': 'Call this to get the weather at that location',
     'type': 'object',
     'properties': {'location': {'title': 'Location',
       'description': 'location to get weather for',
       'type': 'string'}},
     'required': ['location']},
    'description': 'Run weather search.'},
   'type': 'function'},
  {'function': {'name': 'search_animal',
    'parameters': {'title': 'AnimalSearch',
     'description': 'Call this to get the animal',
     'type': 'object',
     'properties': {'animal': {'title': 'Animal',
       'description': 'animal that is mentioned',
       'type': '

In [46]:
def search_weather(location: str) -> str:
    """Run weather search."""
    return "i am weather bob" #just a simple one. just to test

In [47]:
def search_animal(animal: str) -> str:
    """Run animal search."""
    return "i am a pink panther" #just a simple one. just to test

In [48]:
available_functions = {
    "search_weather": search_weather,
    "search_animal": search_animal
}

In [49]:
available_functions

{'search_weather': <function __main__.search_weather(location: str) -> str>,
 'search_animal': <function __main__.search_animal(animal: str) -> str>}

In [51]:
functions = [
    {
        "name": "search_weather",
        "description": "Run weather search.",
        "parameters": {
            "title": "WeatherSearch",
            "description": "Call this to get the weather at that location",
            "type": "object",
            "properties": {
                "location": {
                    "title": "Location",
                    "description": "location to get weather for",
                    "type": "string"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "search_animal",
        "description": "Run animal search.",
        "parameters": {
            "title": "AnimalSearch",
            "description": "Call this to get the animal",
            "type": "object",
            "properties": {
                "animal": {
                    "title": "Animal",
                    "description": "animal that is mentioned",
                    "type": "string"
                }
            },
            "required": ["animal"]
        }
    }
]

In [58]:
def execute_function_call(function_name, arguments):

    # Retrieve the function object using the function name
    function = available_functions.get(function_name, None)
    
    if function:
        # Call the function object with the arguments
        results = function(**arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    
    return results

The above function is important. You need it to execute the chosen function. (Chosen function means the function that the model calls/chooses to execute and get results from. 
- model will choose which function to call
- after it has chosen, it will wait for YOUR ACTION
- Your action is to execute the 'chosen' function and arguments. 


### What does a monkey eat? (Attempt 1)

In [52]:
thread, run = create_thread_and_run(
    "What does a monkey eat?"
)
run = wait_on_run(run, thread)
run.status

'requires_action'

In [53]:
show_json(run)

{'id': 'run_MoUoPUBrf7bLWBkix3z0b0T1',
 'assistant_id': 'asst_b11XnU3yNtM3CHOOoNKNYOvt',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702883215,
 'expires_at': 1702883815,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a chatty bot who can search weather and search animal. Entertain the user.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_FGeyMadxv5C8Oh7reRYuCDlf',
     'function': {'arguments': '{"animal":"monkey"}', 'name': 'search_animal'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'started_at': 1702883216,
 'status': 'requires_action',
 'thread_id': 'thread_a046oX3s1rOmZ6r54lcASoTq',
 'tools': [{'function': {'name': 'search_weather',
    'parameters': {'title': 'WeatherSearch',
     'description': 'Call this to get the weather at that location',
     'type': 'object',
     'properties': {'location': {'title'

The 'required_action' field --> `'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_4ikV54J3AGKYwBVCsdLwesU8',
     'function': {'arguments': '{"animal":"elephant"}',
      'name': 'search_animal'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'started_at': 1702869449,
 'status': 'requires_action',
`

field indicates a Tool is waiting for us to run it and submit its output back to the Assistant.
It is calling for the 'search_animal' function. You need to parse the 'name' and 'arguments'.

#### Lets check out the chosen function_name and the arguments

In [54]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
function_name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", function_name)
print("Function Arguments:")
arguments

Function Name: search_animal
Function Arguments:


{'animal': 'monkey'}

In [55]:
arguments

{'animal': 'monkey'}

In [56]:
function_name

'search_animal'

#### Run 'execute_function_call' to get the result of the function

In [59]:
function_response = execute_function_call(function_name, arguments)
function_response

'i am a pink panther'

In [60]:
json.dumps(function_response)

'"i am a pink panther"'

In [61]:
run.status

'requires_action'

In [65]:
function_response

'i am a pink panther'

#### Submit these arguments to the Assistant. 
We'll need the `tool_call` ID, found in the `tool_call` we parsed out earlier. We'll also need to encode our `list`of responses into a `str`.

In [62]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(function_response),
        }
    ],
)
#show_json(run)

In [63]:
run.status

'queued'

In [64]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What does a monkey eat?
assistant: Oops! Looks like there's been a mix-up. Let me try that again.



Weird? The Assistant should have replied with a fixed *function response* that says "i am a pink panther...". However, it did not return this response. I assume because it first determines whether the response is appropriate with the question asked by the User: "What does a monkey eat?".
When the Assistant found that the *'function response' answer* is not relevant to the question, it will come up with some other response. (Probably it is its fallback mechanism?)

### What does a monkey eat? (Attempt 2)

In attempt to observe whether the Assistant accepts the function response, lets try to input a relevant (hard-coded) answer.

In [79]:
thread, run = create_thread_and_run(
    "What does a monkey eat?"
)
run = wait_on_run(run, thread)
run.status

'requires_action'

In [80]:
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
function_name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

In [81]:
function_response = execute_function_call(function_name, arguments)
function_response

'i am a pink panther'

Lets cheat and override that function response :p I want the answer to be food-related so that it is relevant to the asked question.

In [82]:
function_response = "banana and flower"

In [83]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(function_response),
        }
    ],
)
#show_json(run)

In [84]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What does a monkey eat?
assistant: Monkeys eat bananas and flowers! They have quite a colorful diet, don't they? If you'd like to know more about monkeys or any other animals, just let me know!



p/s: The judgement might be incorrect but thats it for this notebook. Thanks!